# import lib and test GPU of tensorflow


In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
# import matplotlib.image as mping
import matplotlib.animation as animation
import time, datetime
import sys
import gc
import os
import tensorflow as tf
import h5py
from random import randint
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import manifold, datasets
from IPython.display import clear_output
from tsnecuda import TSNE
#import tsnecuda
#tsnecuda.test()


In [2]:
# os.environ["QT_API"] = "pyqt5"
# print(os.environ.get('QT_API'))
import tsnecuda
print(tsnecuda.__file__)


/home/shawn/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tsnecuda/__init__.py


In [3]:
%matplotlib qt


In [4]:
#test tesnsorflow
print("tensorflow version: ",tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))
# Returns whether TensorFlow was built with CUDA (GPU) support.
print("tf.test.is_Built_With_cuda = ",tf.test.is_built_with_cuda())
# RetUrns whether TensorFlow can access a GPU. (deprecated)
print("tf.test.is_gpu_available = ",tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))


tensorflow version:  2.1.0
Num GPUs Available:  1
Num CPUs Available:  1
tf.test.is_Built_With_cuda =  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


tf.test.is_gpu_available =  True


# Main program -------------->
# read data(.npy)


In [27]:
def read_data_and_build_data_label():
    folder_path = '/home/shawn/DATA/Special_topic/Pleurotus_eryngii_complete/spectral_curve_v1/A/*' # Shawn Linux

    file_name = sorted(glob.glob(folder_path))
    # print('\n'.join(file_name))
    data_n = len(file_name)
    # print(data_n)

    ## 0511 --> 0
    ## 0601 --> 8
    label_value = 0
    data_label = np.zeros(data_n, dtype=np.int8)
    check = file_name[0][78:82]
    print(check)
    for i in range(data_n):
        if file_name[i][78:82] != check:
            # print(check)
            label_value += 1
            check = file_name[i][78:82]
        data_label[i] = label_value
    print(data_label)

    # # check label
    # for i in range(data_n):
    #     print("{}\t{}".format(data_label[i], file_name[i][-16:-12]))

    data = np.zeros((data_n,300))
    for i in range(data_n):
        data[i] = np.load(file_name[i])

    print("read data successfull")
    return data_n, data_label, data

data_n, data_label, data = read_data_and_build_data_label()


0511
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7]
read data successfull


# z-scroce


In [28]:
# z-scroce
print("total is {}".format(data_n))
for data_i in range(data_n):
    m = np.mean(data[data_i])
    std = np.std(data[data_i])
    data[data_i] = (data[data_i]-m)/std
print("sucessful")

total is 159
sucessful


# select data


In [29]:
print(data.shape)
data_v1 = np.delete(data, np.s_[270:300], 1)
data_v1 = np.delete(data_v1, np.s_[0:30], 1)
print(data_v1.shape)


(159, 300)
(159, 240)


In [26]:
def show_spectral_curve(data_label, data):
    data_band_n = data_v1.shape[1]
    print(data_band_n)

    colormap = plt.cm.rainbow(np.linspace(0, 1, data_label[-1]+1))
    print(colormap.shape)

    plt.clf()
    plt.ion()

    legend_elements = [Line2D([0], [0], marker='o', color='w', label='0511',
                              markerfacecolor=colormap[0], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0514',
                              markerfacecolor=colormap[1], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0516',
                              markerfacecolor=colormap[2], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0521',
                              markerfacecolor=colormap[3], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0523',
                              markerfacecolor=colormap[4], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0528',
                              markerfacecolor=colormap[5], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0530',
                              markerfacecolor=colormap[6], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0601',
                              markerfacecolor=colormap[7], markersize=15)]
    plt.legend(handles=legend_elements)

    plt.xlim(-10,300)
    plt.ylim(-3,1.5)
    plt.xticks(np.arange(0, 300, 15))

    for i in range(data_n):
        plt.plot(np.arange(30,data_band_n+30,1),data[i],color=colormap[data_label[i]])
        # plt.pause(0.001)

    plt.show()

show_spectral_curve(data_label, data_v1)

240
(8, 4)


# t-SNE


In [33]:
n_components = 2
perplexity = 11
learning_rate = 150
n_iter = 3000

early_exaggeration = 20
n_iter_without_progress = 300
min_grad_norm = 1e-3
metric = 'euclidean'
init = 'random'
verbose = 1

per_s = 15
per_e = 25

data_embedded_arr = np.zeros((per_e-per_s,data_n,2),np.float32)
print('perplexity interval = {}-{}'.format(per_s, per_e-1))
ST = datetime.datetime.now()

for per_i in range(per_s, per_e):
    print('{}/{}'.format(per_i, per_e-1),end='\t')
    data_embedded_arr[per_i-per_s] = TSNE(perplexity=per_i, learning_rate=learning_rate, n_iter=n_iter).fit_transform(data_v1)

ET = datetime.datetime.now()
print("\nExec.:",ET-ST)
print("t-sne successfull")
    # data_embedded = TSNE(perplexity=perplexity, learning_rate=learning_rate, n_iter=n_iter).fit_transform(data)



perplexity interval = 15-24
15/24	

16/24	

17/24	

18/24	

19/24	

...

21/24	

22/24	

23/24	

24/24	


Exec.: 0:00:35.233588
t-sne successfull


In [34]:
plt.clf()

In [35]:
# show data
colormap = plt.cm.rainbow(np.linspace(0, 1, data_label[-1]+1))

rotation=[1,4,5,6,3,2]
for per_i in range(per_s, per_e):
    plt.subplot(2,3,rotation[(per_i-per_s)%6])
    plt.cla()

    legend_elements = [Line2D([0], [0], marker='o', color='w', label='0511',
                              markerfacecolor=colormap[0], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0514',
                              markerfacecolor=colormap[1], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0516',
                              markerfacecolor=colormap[2], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0521',
                              markerfacecolor=colormap[3], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0523',
                              markerfacecolor=colormap[4], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0528',
                              markerfacecolor=colormap[5], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0530',
                              markerfacecolor=colormap[6], markersize=15),
                       Line2D([0], [0], marker='o', color='w', label='0601',
                              markerfacecolor=colormap[7], markersize=15)]

    plt.legend(handles=legend_elements)
    for i in range(data_n):
        plt.title('perplexity='+str(per_i))
        plt.scatter(data_embedded_arr[per_i-per_s][i][0], data_embedded_arr[per_i-per_s][i][1], color=colormap[data_label[i]])
    plt.show
    plt.pause(0.5)

/home/shawn/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:6: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  


In [ ]:
print(data_embedded_arr.shape)
print(type(data_embedded_arr[0][0][0]))

# test


In [ ]:
print(data[1].shape)
print(data.shape)
plt.plot(np.arange(0,300,dtype=np.int16), data[3])